In [ ]:
import json
# For prerequisites running the following sample, visit https://help.aliyun.com/document_detail/611472.html
from http import HTTPStatus
import dashscope

dashscope.api_key = 'sk-ff167cad89364e2491fc6193e006c4a3'


def call_with_messages(temperature, messages):
    response = dashscope.Generation.call(
        model='llama3-8b-instruct',
        messages=messages,
        result_format='message',  # set the result to be "message" format.
        temperature=temperature,
        top_p=0.95
    )
    if response.status_code == HTTPStatus.OK:
        return response.output
    else:
        return ""


def extract_code_from_content(language, content):
    start = content.find("```")
    if start == -1:
        return content

    end = content.find("```", start + 3)
    if end == -1:
        return content

    code_block = content[start + 3:end].strip()

    first_newline = code_block.find("\n")
    if first_newline != -1:
        code_block = code_block[first_newline:].strip()
    if language != "c&cpp":
        return code_block
    elif language == "c&cpp":
        return code_block.split("int main()")[0]
    return code_block

In [ ]:
from tqdm import tqdm

LANGUAGE_LIST = ['c&cpp']
QUESTION_PATH = r"E:\code\code_back\python_project\llm\qa\question\all_question.json"
PASS_NUMBER = 1
temperature = 0
if PASS_NUMBER == 1:
    temperature = 0.01
elif PASS_NUMBER == 10:
    temperature = 0.8
MODEL_NAME = "Meta-Llama-3.1-8B-Instruct"
ANSWER_PATH = rf"E:\code\code_back\python_project\llm\qa\{MODEL_NAME}_answer"
for language in LANGUAGE_LIST:
    with open(QUESTION_PATH, "r", encoding="utf8") as file:
        question_list = json.load(file)
        for question in tqdm(question_list):
            all_response = []
            try:
                prompt = question["language_version_list"][language]['prompt']
                if prompt == "":
                    continue
                messages = [
                    {'role': 'user', 'content': prompt + "give the all code, no examples are required. no main() func required"}
                ]
                for loop in range(1, PASS_NUMBER + 1):
                    response = call_with_messages(temperature, messages)
                    response_code = str(response.choices[0].message.content)
                    all_response.append({"index": loop, "model_name": MODEL_NAME, "response_code": extract_code_from_content(language,response_code)})
                question["language_version_list"][language]["answer_list"] = all_response
                print(question)
                with open(f"{ANSWER_PATH}/{language}_answer_pass{PASS_NUMBER}.jsonl", "a+", encoding='utf8') as file:
                    json_str = json.dumps(question)
                    file.write(json_str + "\n")
                    file.flush()
            except Exception as e:
                print(e)
                continue
            print(f"{question['task_id']} start answer finish")